In [6]:
from IPython.display import display, HTML

display(HTML('''
    <div class="stat-block" style="padding: 1em; border: 1px solid white; width: fit-content;">
        <h1 style="font-family: serif; margin-top: 0em; margin-bottom: 0em;">{0}</h1>
        <small><em>{1}, {2}</em></small><br/>
        <svg height="5" width="100%" class="tapered-rule">
            <polyline points="0,0 400,2.5 0,5" fill="white"></polyline>
        </svg><br/>
        <span><strong>Armor Class</strong> 10 Natural Armor</span><br/>
        <svg height="5" width="100%" class="tapered-rule">
            <polyline points="0,0 400,2.5 0,5" fill="white"></polyline>
        </svg><br/>
        <div style="display: grid; grid-template-columns: repeat(6, 1fr); grid-gap: 1em;">
            <div style="text-align: center;"><strong>STR</strong><br>10 (+0)</div>
            <div style="text-align: center;"><strong>DEX</strong><br>10 (+0)</div>
            <div style="text-align: center;"><strong>CON</strong><br>10 (+0)</div>
            <div style="text-align: center;"><strong>INT</strong><br>10 (+0)</div>
            <div style="text-align: center;"><strong>WIS</strong><br>10 (+0)</div>
            <div style="text-align: center;"><strong>CHA</strong><br>10 (+0)</div>
        </div>
        <svg height="5" width="100%" class="tapered-rule"> 
            <polyline points="0,0 400,2.5 0,5" fill="white"></polyline>
        </svg>
    </div>
'''.format("Rat", "small humanoid", "any alignment")))


In [1]:
from xander.engine import dice
from xander.engine.dice import DExpr, D20, D100

dice.set_seed(0)

True

In [ ]:
from xander.engine import character
character.load_from_json("rat.json")